Diferensiasi menggunakan Autograd
=======================================
PyTorch mempunyai mekanisme diferensiasi bawaan yang disebut dengan ``torch.autograd``. Mari kita lihat bagaimana ``autograd`` mengumpulkan gradien. Buat 2 tensor ``a`` dan ``b`` dengan
``requires_grad=True``. Ini memberi arahan bagi ``autograd`` bahwa setiap operasi terhadap tensor ini harus dilacak.




In [17]:
import torch

x = torch.rand(5, 5)
print(x)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

tensor([[0.3165, 0.5788, 0.7366, 0.4825, 0.4365],
        [0.9940, 0.1766, 0.8471, 0.7569, 0.0680],
        [0.9398, 0.0543, 0.8950, 0.8179, 0.3507],
        [0.3545, 0.0845, 0.1134, 0.1786, 0.2388],
        [0.1492, 0.1998, 0.3847, 0.3899, 0.6460]])
Does `a` require gradients? : False
Does `b` require gradients?: True


In [18]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
print(a)
print(b)

tensor([2., 3.], requires_grad=True)
tensor([6., 4.], requires_grad=True)


Kita membuat tensor lain ``Q`` dari ``a`` dan ``b``.

\begin{align}Q = 3a^3 - b^2\end{align}



In [19]:
Q = 3*a**3 - b**2
Q

tensor([-12.,  65.], grad_fn=<SubBackward0>)

Aumsikan ``a`` dan ``b`` sebagai parameter dari NN, dan ``Q`` eror. Di proses pelatihan NN, kita ingin gradien dari eror terhadap parameter, dimana: 

\begin{align}\frac{\partial Q}{\partial a} = 9a^2\end{align}

\begin{align}\frac{\partial Q}{\partial b} = -2b\end{align}


Ketika kita memanggil ``.backward()`` pada ``Q``, autograd menghitung gradien ini dan. menyimpannya dalam atribut ``.grad`` dari tensor tersebut.

Kita harus menggunakan argumen ``gradient`` di ``Q.backward()`` karena ini adalah sebuah vektor. ``gradient`` adalah sebuah tensor dari bentuk yang sama dengan ``Q``, dan merepresentasikan gradien dari Q terhadap dirinya sendiri, dimana:

\begin{align}\frac{dQ}{dQ} = 1\end{align}

Kita dapat pula agregasi Q menjadi sebuah skalar dan memanggil backward secara implisit, seperti ``Q.sum().backward()``.




In [20]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

Gradien sekarang telah disimpan dalam ``a.grad`` dan ``b.grad``



In [21]:
# cek apakah gradien yang tersimpan terhitung benar
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


Soal
----------
Buatlah loss function sebagai berikut:
\begin{align}Y = \Sigma  ln(x) \end{align}

Gunakan autograd untuk menghitung gradiennya terhadap parameter x

Hitung turunannya secara analitik dan bandingkan

In [22]:
x = torch.tensor([1., 2., 3., 4., 5.], requires_grad=True)
Y = torch.sum(torch.log(x))
# backward pass
Y.backward()
x.grad

tensor([1.0000, 0.5000, 0.3333, 0.2500, 0.2000])

Diferensiasi Neural Network Otomatis melalui ``torch.autograd``
=======================================

Algoritma untuk melatih neural networks adalah **back propagation**. Pada algoritma ini, parameters (bobot) akan diubah sesuai dengan **gradient** dari loss function terhadap parameter tersebut.

Untuk menghitung gradient itu, PyTorch mempunyai mekanisme diferensiasi bawaan yang disebut dengan ``torch.autograd``. Ini mempermudah perhitungan gradien secara otomatis terhadap computational graph jenis apapun.

Cobalah 1-layer neural network, dengan input  ``x``,parameters ``w`` and ``b``, dan loss function sebagai berikut:


In [23]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
loss

tensor(0.9513, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

Soal
-----

Hitung binary cross entropy alternatif dengan menggunakan operasi tensor

In [24]:
def BCE(x, y):
    p = -y*(torch.log(torch.sigmoid(x))-(1-y)*torch.log(1-torch.sigmoid(x)))
    return p.mean()

loss_manual = BCE(x, y)
loss_manual

RuntimeError: The size of tensor a (3) must match the size of tensor b (5) at non-singleton dimension 0

Tensor dan Gradien
------------------------------------------

pada network ini, ``w`` dan ``b`` adalah **parameters**, yang akan kita optimalisasi. 
Maka, kita memerlukan perhitungan gradien dari sebuah loss function terhadap variabel-variabel tersebut. Untuk bisa melakukannya, kita mengatur ``requires_grad`` dari tensor-tensor tersebut.



<div class="alert alert-info"><h4>Catatan</h4><p>Kamu dapat mengatur ``requires_grad`` ketika menciptakan sebuah tensor, atau mengaturnya nanti dengan menggunakan metode ``x.requires_grad_(True)``.</p></div>



Sebuah fungsi yang kita aplikasikan ke tensor sebagai kontruksi computational graph adalah sebuah kelas objek ``Function``. Objek ini mengerti bagaimana cara menghitung fungsi dalam arah *forward*, dan juga bagaimana cara menghitung turunannya selama langkah-langkah *backward propagation*. Sebuah referensi terhadap fungsi backward propagation disimpan dalam properti ``grad_fn`` dari sebuah tensor. Kamu dapat menemukan informasi lebih lanjut mengenai ``Function`` dalam dokumentasi berikut
https://pytorch.org/docs/stable/autograd.html#function


In [25]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x0000027D8D9AFB80>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x0000027D84608D90>


Menghitung Gradien
-------------------

Untuk optimalisasi bobot pada neural network, kita perlu menghitung turunan dari loss function terhadap parameter. Secara khusus, kita memerlukan $\frac{\partial loss}{\partial w}$ dan
$\frac{\partial loss}{\partial b}$ terhadap nilai dari ``x`` and ``y``. Untuk menghitung turunan itu, kita memanggil ``loss.backward()``, lalu mendapatkan nilai ``w.grad`` dan
``b.grad``:



In [26]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2903, 0.1557, 0.0541],
        [0.2903, 0.1557, 0.0541],
        [0.2903, 0.1557, 0.0541],
        [0.2903, 0.1557, 0.0541],
        [0.2903, 0.1557, 0.0541]])
tensor([0.2903, 0.1557, 0.0541])


<div class="alert alert-info"><h4>Catatan</h4><p>- Kita akan memperoleh properti ``grad`` untuk leaf node dari computational graph, dimana properti ``requires_grad`` diatur menjadi ``True``. Untuk semua nodes yang lain di graph ini, gradien tidak tersedia. 
    - Kita hanya dapat melakukan penghitungan gradien menggunakan ``backward`` sekali pada graph tersebut, untuk alasan performa. Jika kita membutuhkan memanggil beberapa ``backward`` pada graph yang sama, kita perlu melakukan ``retain_graph=True`` to the ``backward`` call.</p></div>


Mematikan Pelacakan Gradien
---------------------------

Secara bawaan, semua tensor dengan ``requires_grad=True`` akan melacak sejarah komputasi dan mendukung komputasi gradien. Tetapi, terdapat beberapa kasus dimana kita tidak perlu melakukannya. Misalnya ketika kita melatih model dan hanya ingin menggunakannya terhadap beberapa input data, kita hanya perlu melakukan komputasi *forward*. Kita bisa berhenti melacak komputasi dengan menggunakan blok ``torch.no_grad()``.



In [27]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


Cara lain untuk memperoleh hasil yang sama adalah dengan menggunakan metode ``detach()`` pada tensor:




In [28]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


Ada alasan lain untuk mematikan pelacakan gradien:
    - Untuk menandai beberapa parameter di neural network sebagai **frozen parameters**. Ini skenario yang sangat umum untuk `finetuning a pretrained network` https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
    - Untuk mempercepat perhitungan ketika hanya menghitung gerakan forward, karena komputasi pada tensors tanpa pelacakan gradien akan jauh lebih efisien
    

Tambahan untuk Computational Graphs
----------------------------
Autograd menyimpan catatan data (tensors) dan semua operasi yanbg tereksekusi (bersama dengan hasil tensor yang baru) di sebuah directed acyclic graph (DAG) yang berisi objek `Function` https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function. Pada DAG ini, leaves adalah input tensor, root adalah output tensor. Dengan melacak jejaring ini dari roots ke leaves, kamu dapat secara otomatis menghitung gradien dengan menggunakan aturan rantai.

Di sebuah gerakan forward, autograd melakukan 2 hal sekaligus:
- menjalankan operasi untuk menghitung hasil tensor
- menjaga operasi *gradient function* pada DAG

Gerakan backward dimulai ketika ``.backward()`` dipanggil pada DAG root. 
``autograd`` kemudian:
- menghitung gradien dari setiap ``.grad_fn``
- mengakumulasi semuanya pada tensor dengan atribut ``.grad``
- menggunakan aturan rantai, melakukan propagasi semuanya sampai kepada tensor leaf.

<div class="alert alert-info"><h4>Catatan</h4><p>**DAGs bersifat dinamis di PyTorch**
    Sebuah hal penting untuk diperhatikan adalah jejaring yang diciptakan dari kosong; setelah masing-masing pemanggilan ``.backward()``, autograd mulai mengisi jejaring yang baru. Ini yang mengijinkanmu menggunakan control flow statements pada modelmu; kamu dapat merubah bentuk, ukuran, dan operasi pada setiap iterasi jika diperlukan
</p></div>


Soal
-----


Gunakan pretrained resnet18 dari torchvision. Kita akan membuat tensor berisi random data sebagai representasi dari 1 gambar dengan 3 chanel, tinggi & lebarnya adalah 64, dan labelnya diinisiasi dengan secara random. Labelnya berbentuk (1,1000). 



In [29]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)


Lakukan prediksi label berdasarkan model dan data


In [30]:
# forward pass
prediksi = model(data)
prediksi

tensor([[-5.9653e-01, -4.7343e-01, -3.9114e-01, -1.3775e+00, -5.3189e-01,
          7.5216e-02, -4.0485e-01,  2.3898e-02,  1.4629e-01, -8.6787e-01,
         -1.2443e+00, -1.0371e+00, -5.2396e-01, -1.2504e+00, -1.0534e+00,
         -1.0168e+00, -8.5270e-01, -4.5728e-01, -5.9357e-01, -5.3287e-01,
         -1.5239e+00, -8.1629e-01, -1.6670e+00, -8.7941e-02, -1.1051e+00,
         -1.0063e+00, -6.8265e-01, -1.1368e+00, -6.0703e-01, -2.4777e-01,
         -4.8513e-01, -7.0539e-01, -2.9475e-01, -1.9521e-02,  1.5139e-01,
         -6.2433e-02,  9.9152e-01, -3.9921e-01, -2.6203e-01,  1.8758e-01,
         -5.8577e-01, -5.5779e-01, -1.0074e+00, -3.4122e-01, -2.3087e-01,
          6.3094e-03, -5.0163e-01, -2.1721e-01, -1.0577e+00, -7.3264e-01,
          1.2978e-01,  5.2467e-01, -2.7880e-02, -4.6113e-01,  7.8067e-02,
         -8.9861e-01, -9.1839e-02, -1.1714e+00, -6.1436e-02, -2.4839e-01,
          1.0792e+00,  6.1289e-01,  3.3999e-01,  3.5901e-01, -2.9910e-01,
          8.2453e-02,  7.8442e-03, -6.

Setelah itu hitung loss function sebagai total perbedaan prediksi terhadap label dan lakukan backpropagation sekali


In [14]:
# loss function

# backward pass


tensor(1034.3706, grad_fn=<SumBackward0>)


Gunakan optimisasi SGD dengan menggunakan learning rate 1e-7 dan momentum 0.9

In [15]:
optim = torch.optim.SGD(model.parameters(), lr=1e-7, momentum=0.9)

Panggil ``.step()`` untuk menginisiasi gradient descent. Optimalisasi akan mengubah parameter sesuai dengan gradien yang telah tersimpan di ``.grad``.


In [16]:
optim.step() #gradient descent

Prediksi dan hitung akurasi model

In [17]:
prediksi = model(data)
loss = ((prediksi - labels)**2).sum()
print(loss)
loss.backward()
optim.step()

tensor(1008.0511, grad_fn=<SumBackward0>)


Coba sekarang bekukan semua parameter dari model tersebut

In [ ]:
for param in model.parameters():
    param.requires_grad = False

Prediksi dan hitung akurasi model

Bacaan Opsional: Gradien Tensor dan Produk Jacobian
--------------------------------------

Di banyak kasus, kita memiliki loss function skalar, dan perlu menghitung gradien terhadap suatu parameter. Tetapi ada kasus dimana fungsi output adalah sebuah tensor. Pada kasus ini, PyTorch mengijinkan perhitungan **Jacobian product**, dan bukan gradien tersebut.

Pada sebuah fungsi vektor $\vec{y}=f(\vec{x})$, dimana
$\vec{x}=\langle x_1,\dots,x_n\rangle$ dan
$\vec{y}=\langle y_1,\dots,y_m\rangle$, sebuah gradien dari
$\vec{y}$ terhadap $\vec{x}$ adalah **Jacobian
matrix**:

\begin{align}J=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}

Daripada menghitung matriks Jacobian tersebut, PyTorch mengijinkan perhitungan **Jacobian Product** $v^T\cdot J$ untuk sebuah vektor input $v=(v_1 \dots v_m)$. Ini dapat dicapai dengan memanggil ``backward`` dengan $v$ sebagai argumen. Ukuran $v$ harus sama dengan ukuran tensor awal, yang terhadapnya kita ingin menghitung produk:




In [ ]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


Perhatikan ketika kita memanggil ``backward`` untuk kedua kali dengan argumen yang sama, nilai dari gradien berbeda. Ini terjadi karena ketika melakukan propagasi ``backward``, PyTorch **mengakumulasi gradien**, dimana nilai gradien yang terhitung ditambahkan kepada properti ``grad`` dari semua leaf nodes dari computational graph. Jika kamu ingin menghitung gradien sebenarnya, kamu perlu mengosongkan properti ``grad`` sebelumnya. Di dalam pelatihan, sebuah *optimizer* membantu kita untuk melakukan hal ini.



<div class="alert alert-info"><h4>Catatan</h4><p>Sebelumnya kita memanggil fungsi ``backward()`` tanpa parameter. Ini ekuivalen dengan memanggil ``backward(torch.tensor(1.0))``, dimana ini cara yang efektif untuk menghitung gradien dalam kasus fungsi skalar, seperti loss dalam pelatihan neural network
</p></div>




--------------


